In [1]:
#### t_fwd3 vs t_fwd on train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
#from model_triton import *
BS, H, N, D = 2, 2, 512, 64 # 1, 1, 3, 4 #2, 2, 5, 4
vocab_size = 128
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda").to(torch.int32)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
# print(mask)

from model_triton import t_gpt2_forward, t_gpt2_forward_with_acts, t_gpt2_forward_with_acts_t
y_in = y[:, :-1]
y_out = y[:, 1:]
    
res2 = t_gpt2_forward(layers_params, y_in, mask, None, True, p_gen_aux)
print(res2)
#print(res2[1])
#print_res_shapes(res2[0]) 

print(f'----XXX----')

res3, _ = t_gpt2_forward_with_acts(layers_params, y_in, mask, None, True, p_gen_aux)
print(res3)
#print_res_shapes(res3[0]) 


tensor([[[ 0.036103,  0.136018,  0.178939,  ...,  0.037634, -0.097982,
           0.065277],
         [ 0.019600,  0.068034,  0.001007,  ...,  0.193625,  0.062375,
           0.247736],
         [ 0.168226, -0.072604,  0.064664,  ...,  0.120566,  0.173300,
           0.078101],
         ...,
         [ 0.183725, -0.112369, -0.153559,  ..., -0.112961, -0.226167,
          -0.071344],
         [-0.198194, -0.044662,  0.175700,  ...,  0.241721, -0.198204,
           0.137451],
         [ 0.099250, -0.099887, -0.101990,  ...,  0.043918, -0.190028,
           0.108264]],

        [[ 0.042967, -0.046060,  0.018442,  ..., -0.170827,  0.189597,
          -0.187806],
         [ 0.260121, -0.364823,  0.160448,  ..., -0.072764,  0.226546,
           0.264742],
         [ 0.106135, -0.116928,  0.024084,  ..., -0.068344, -0.101456,
          -0.053335],
         ...,
         [ 0.035734,  0.065116,  0.017479,  ...,  0.089050, -0.023398,
           0.081217],
         [-0.138957,  0.277459, -0.18751